# Build my own deep learning functions

## Purpose

Learn by practicing. Notations follow Andrew Ng's Coursera deep learning course.

### Build the forward_prop function

Note: If there is error while importing python modules while running this notebook in vscode, make sure the both the vscode python interpreter and ipython kernal are both set properly. 

In [1]:
import numpy as np

In [2]:
def forward_prop(X, Y, parameters):
    '''
    # TODO: What's the row # variable?
    X: Input data. (n0???, m). n0: feature #; m: # of examples
    Y: Labels. (1, m)
    parameters: The model parameters

    J: the cost
    '''
    L = len(parameters) // 2
    for l in range(1, L):
        # print('l=', l)
        W = parameters['W'+str(l)]
        b = parameters['b'+str(l)]

        assert(W.shape[0] == b.shape[0])
        Z = np.dot(W, X) + b
    
    J = 0.0

    return J


In [3]:
parameters = {}
parameters['W1'] = 1.1
parameters['b1'] = 1.2
parameters['W2'] = 2.1
parameters['b2'] = 2.2
parameters['W3'] = 3.1
parameters['b3'] = 3.2
forward_prop(0, 0, parameters)


3